Loading data 

In [17]:
# Section must be included at the beginning of each new notebook. Remember to change the app name. 
# If you're using VirtualBox, change the below to '/home/user/spark-2.1.1-bin-hadoop2.7'
import findspark 
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('basics').getOrCreate()

In [18]:
# Let's read in the data. If you open the dataset, you'll find that each column has a header. We specify that by stating that header=True.
# To make our lives easier, we can also use 'inferSchema' when importing CSVs. This automatically detects data types.
# If you would like to manually change data types, refer to this article: https://medium.com/@mrpowers/adding-structtype-columns-to-spark-dataframes-b44125409803
df = spark.read.csv('AppleStore.csv',header=True,inferSchema=True)

In [6]:
# The show method allows you visualise DataFrames in a tabular format. 
df.show()

NameError: name 'df' is not defined

In [20]:
# Print schema allows us to visualise the data structure at a high level. 
df.printSchema()

# We can also use head to print a specific amount of rows, so we can get a better understanding of the data points. 
# Note that we have to specify 'print' depending on the method we're using. Otherwise it may not show up!
print(df.head(1))

root
 |-- _c0: integer (nullable = true)
 |-- id: integer (nullable = true)
 |-- track_name: string (nullable = true)
 |-- size_bytes: long (nullable = true)
 |-- currency: string (nullable = true)
 |-- price: double (nullable = true)
 |-- rating_count_tot: integer (nullable = true)
 |-- rating_count_ver: integer (nullable = true)
 |-- user_rating: double (nullable = true)
 |-- user_rating_ver: double (nullable = true)
 |-- ver: string (nullable = true)
 |-- cont_rating: string (nullable = true)
 |-- prime_genre: string (nullable = true)
 |-- sup_devices.num: integer (nullable = true)
 |-- ipadSc_urls.num: integer (nullable = true)
 |-- lang.num: integer (nullable = true)
 |-- vpp_lic: integer (nullable = true)

[Row(_c0=1, id=281656475, track_name='PAC-MAN Premium', size_bytes=100788224, currency='USD', price=3.99, rating_count_tot=21292, rating_count_ver=26, user_rating=4.0, user_rating_ver=4.5, ver='6.3.5', cont_rating='4+', prime_genre='Games', sup_devices.num=38, ipadSc_urls.num=5

In [41]:
# Let's select the columns that are integers, and use the describe method again.
# We see that the average age is 41. The average bank account balance is $1,074. 
# And they spoke to call centre reps for approx. 931 seconds on average. 
df.select('id','size_bytes','currency','price','rating_count_tot','rating_count_ver','user_rating','user_rating_ver','ver','cont_rating','prime_genre','vpp_lic').describe().show()

+-------+--------------------+-------------------+--------+------------------+------------------+------------------+-----------------+-----------------+------------------+-----------+-----------+-------------------+
|summary|                  id|         size_bytes|currency|             price|  rating_count_tot|  rating_count_ver|      user_rating|  user_rating_ver|               ver|cont_rating|prime_genre|            vpp_lic|
+-------+--------------------+-------------------+--------+------------------+------------------+------------------+-----------------+-----------------+------------------+-----------+-----------+-------------------+
|  count|                7197|               7197|    7197|              7197|              7197|              7197|             7197|             7197|              7197|       7197|       7197|               7197|
|   mean| 8.631309974515771E8| 1.99134453825066E8|    null|1.7262178685562626|12892.907183548701| 460.3739057940809|3.526955675976101|3.

In [43]:
# Let's select the balance column and assign it to a variable. 
price_col = df.select('price')

# We can then use the show method on that variable.
price_col.show()

+-----+
|price|
+-----+
| 3.99|
|  0.0|
|  0.0|
|  0.0|
|  0.0|
| 0.99|
|  0.0|
|  0.0|
| 9.99|
| 3.99|
| 4.99|
| 7.99|
|  0.0|
|  0.0|
| 4.99|
|  0.0|
|  0.0|
|  0.0|
|  0.0|
| 2.99|
+-----+
only showing top 20 rows



In [46]:
# We can also add columns and manipulate the DataFrame. Let's times balance by 10, and add the output to a new column.
df.withColumn('price_times_10',df['price']*10).show()
# Question: If we print the df DataFrame again, why is the 'balance_times_10 column' missing?
df.show()


+---+---------+--------------------+----------+--------+-----+----------------+----------------+-----------+---------------+-------+-----------+-----------------+---------------+---------------+--------+-------+------------------+
|_c0|       id|          track_name|size_bytes|currency|price|rating_count_tot|rating_count_ver|user_rating|user_rating_ver|    ver|cont_rating|      prime_genre|sup_devices.num|ipadSc_urls.num|lang.num|vpp_lic|    price_times_10|
+---+---------+--------------------+----------+--------+-----+----------------+----------------+-----------+---------------+-------+-----------+-----------------+---------------+---------------+--------+-------+------------------+
|  1|281656475|     PAC-MAN Premium| 100788224|     USD| 3.99|           21292|              26|        4.0|            4.5|  6.3.5|         4+|            Games|             38|              5|      10|      1|39.900000000000006|
|  2|281796108|Evernote - stay o...| 158578688|     USD|  0.0|          1610

In [50]:
# Let's try out some additional DataFrame methods.
# How would we identify individuals with a balance above $5,000? Using filter! 
df.filter("price > 5").show()

# We can also use more advanced filters. For example, let's see the jobs of people with over $2,500 in their bank account.
df.filter("price > 50").select('user_rating','prime_genre').show()


+---+---------+--------------------+----------+--------+------+----------------+----------------+-----------+---------------+-------+-----------+------------+---------------+---------------+--------+-------+
|_c0|       id|          track_name|size_bytes|currency| price|rating_count_tot|rating_count_ver|user_rating|user_rating_ver|    ver|cont_rating| prime_genre|sup_devices.num|ipadSc_urls.num|lang.num|vpp_lic|
+---+---------+--------------------+----------+--------+------+----------------+----------------+-----------+---------------+-------+-----------+------------+---------------+---------------+--------+-------+
|  9|284666222|PCalc - The Best ...|  49250304|     USD|  9.99|            1117|               4|        4.5|            5.0|  3.6.6|         4+|   Utilities|             37|              5|       1|      1|
| 12|284815117|    SCRABBLE Premium| 227547136|     USD|  7.99|          105776|             166|        3.5|            2.5| 5.19.0|         4+|       Games|          

In [52]:
# What if we wanted to identify those that were under 40 and had over $2,500 in their account? 
# We can use multiple conditions.
df.filter("price > 20 AND user_rating < 4.5").select('price','user_rating','prime_genre').show()

+------+-----------+-------------+
| price|user_rating|  prime_genre|
+------+-----------+-------------+
|249.99|        4.0|    Education|
| 74.99|        3.5|   Navigation|
| 24.99|        4.0|    Education|
| 47.99|        0.0|    Reference|
| 24.99|        4.0|        Games|
| 24.99|        4.0|        Music|
| 27.99|        4.0| Food & Drink|
|299.99|        4.0|    Education|
| 49.99|        4.0|        Music|
| 24.99|        1.0|      Medical|
| 20.99|        0.0|   Navigation|
| 39.99|        4.0|        Music|
| 59.99|        4.0|     Business|
| 24.99|        0.0|    Utilities|
| 22.99|        3.5|Photo & Video|
| 22.99|        0.0|    Reference|
| 23.99|        0.0|    Reference|
+------+-----------+-------------+



In [57]:
df.select('price','rating_count_tot','rating_count_ver','user_rating','user_rating_ver','ver','cont_rating','vpp_lic').groupBy('user_rating').mean().show()

+-----------+------------------+---------------------+---------------------+----------------+--------------------+------------------+
|user_rating|        avg(price)|avg(rating_count_tot)|avg(rating_count_ver)|avg(user_rating)|avg(user_rating_ver)|      avg(vpp_lic)|
+-----------+------------------+---------------------+---------------------+----------------+--------------------+------------------+
|        0.0|1.0801291711517784|                  0.0|                  0.0|             0.0|                 0.0|0.9795479009687836|
|        3.5| 1.757435897435901|   15503.266381766382|   138.28774928774928|             3.5|   3.158119658119658|0.9928774928774928|
|        4.5| 1.802110401802427|   22029.743522343222|    803.0623357116035|             4.5|   4.199023657529103|0.9981224183251971|
|        2.5|1.1032142857142866|    5419.933673469388|    50.53061224489796|             2.5|  2.2933673469387754|0.9846938775510204|
|        1.0|1.9265909090909084|   29.568181818181817|   17.20

In [70]:
# To simplify things, let's split this into two steps. First, let's create a variable then order by age.
# Careful when using show()! Otherwise the variable type will change and you won't be able to order it. 
group_prime_genre_df = df.select('price','rating_count_tot','rating_count_ver','user_rating','user_rating_ver','ver','cont_rating','vpp_lic','prime_genre').groupBy('prime_genre').mean()

# Note that we have to use 'avg(age)' instead of age. Why? Because when you use mean(), it changes the feature's name (as you can see below).
print("Sorted by price")
group_prime_genre_df.orderBy('avg(price)').show()


Sorted by price
+-----------------+--------------------+---------------------+---------------------+------------------+--------------------+------------------+
|      prime_genre|          avg(price)|avg(rating_count_tot)|avg(rating_count_ver)|  avg(user_rating)|avg(user_rating_ver)|      avg(vpp_lic)|
+-----------------+--------------------+---------------------+---------------------+------------------+--------------------+------------------+
|         Shopping|0.016311475409836067|    18615.32786885246|    709.0081967213115| 3.540983606557377|  2.6188524590163933|               1.0|
|Social Networking|  0.3398802395209582|    45498.89820359281|    556.2874251497007|2.9850299401197606|  2.4820359281437128|0.9880239520958084|
|          Finance| 0.42115384615384627|   11047.653846153846|   250.57692307692307|2.4326923076923075|  1.6346153846153846|               1.0|
|             News|  0.5177333333333334|   13015.066666666668|               113.72|              2.98|  2.2133333333333

In [73]:
# To simplify things, let's split this into two steps. First, let's create a variable then order by age.
# Careful when using show()! Otherwise the variable type will change and you won't be able to order it. 
group_user_rating_df = df.select('price','rating_count_tot','rating_count_ver','user_rating','user_rating_ver','ver','cont_rating','vpp_lic','prime_genre').groupBy('user_rating').mean()

# Note that we have to use 'avg(age)' instead of age. Why? Because when you use mean(), it changes the feature's name (as you can see below).
print("Sorted by avg(price)")
group_user_rating_df.orderBy('avg(price)').show()

Sorted by avg(price)
+-----------+------------------+---------------------+---------------------+----------------+--------------------+------------------+
|user_rating|        avg(price)|avg(rating_count_tot)|avg(rating_count_ver)|avg(user_rating)|avg(user_rating_ver)|      avg(vpp_lic)|
+-----------+------------------+---------------------+---------------------+----------------+--------------------+------------------+
|        0.0|1.0801291711517784|                  0.0|                  0.0|             0.0|                 0.0|0.9795479009687836|
|        2.5|1.1032142857142866|    5419.933673469388|    50.53061224489796|             2.5|  2.2933673469387754|0.9846938775510204|
|        2.0| 1.155754716981131|    2907.235849056604|   62.075471698113205|             2.0|  1.9858490566037736|0.9905660377358491|
|        1.5|1.2089285714285716|                203.5|   22.482142857142858|             1.5|                1.25|0.9821428571428571|
|        3.0| 1.288067885117496|    7487.

In [105]:
from pyspark.sql.functions import format_number, col

# Let's start off with this. Just grouping by prime_genre and presenting the mean.
group_prime_genre_df = df.select('price','rating_count_tot','rating_count_ver','user_rating','user_rating_ver','ver','cont_rating','vpp_lic','prime_genre').groupBy('prime_genre').mean()
group_prime_genre_df.show()

# Now that we've calculated the mean, the values for blue-collar and technician are extremely long. 
# We can use format_number to reduce the total amount of decimals. 
# The number two represents the amount of decimals we want to be displayed.
group_prime_genre_df = group_prime_genre_df.select('prime_genre',
                                   format_number('avg(price)',2),
                                   format_number('avg(rating_count_tot)',2),
                                   format_number('avg(rating_count_ver)',2),
                                   format_number('avg(user_rating)',2),
                                   format_number('avg(user_rating_ver)',2),
                                   format_number('avg(vpp_lic)',2))

group_prime_genre_df.show()


# But now the column names look quite unprofessional. We can assign an alias to rename each of them.
group_prime_genre_df = group_prime_genre_df.select(col('prime_genre').alias('App Category'),
                                   col('format_number(avg(price), 2)').alias('Average Price'),
                                   col('format_number(avg(rating_count_tot), 2)').alias('Average Total Count'),
                                   col('format_number(avg(rating_count_ver), 2)').alias('Average Version Count'),
                                   col('format_number(avg(user_rating_ver), 2)').alias('Average Version User Rating'),
                                   col('format_number(avg(vpp_lic), 2)').alias('Average Vpp License'))
group_prime_genre_df.show()
                                 
# Finally, let's sort the DataFrame by age.
group_prime_genre_df = group_prime_genre_df.orderBy('Average Price')
print('Average Price, Balance and Duration by App Category')
group_prime_genre_df.show()

+-----------------+--------------------+---------------------+---------------------+------------------+--------------------+------------------+
|      prime_genre|          avg(price)|avg(rating_count_tot)|avg(rating_count_ver)|  avg(user_rating)|avg(user_rating_ver)|      avg(vpp_lic)|
+-----------------+--------------------+---------------------+---------------------+------------------+--------------------+------------------+
|        Education|   4.028233995584995|   2239.2295805739514|                239.0| 3.376379690949227|  2.9525386313465782|               1.0|
|       Navigation|  4.1247826086956545|    11853.95652173913|   110.19565217391305|2.6847826086956523|   2.152173913043478|0.9782608695652174|
|    Entertainment|  0.8897009345794415|    7533.678504672897|   140.89345794392523|3.2467289719626167|                 2.9|0.9981308411214953|
|           Sports|  0.9530701754385958|   14026.929824561403|   128.44736842105263| 2.982456140350877|  2.5482456140350878|0.9473684210

+-----------------+-------------+-------------------+---------------------+---------------------------+-------------------+
|     App Category|Average Price|Average Total Count|Average Version Count|Average Version User Rating|Average Vpp License|
+-----------------+-------------+-------------------+---------------------+---------------------------+-------------------+
|        Education|         4.03|           2,239.23|               239.00|                       2.95|               1.00|
|       Navigation|         4.12|          11,853.96|               110.20|                       2.15|               0.98|
|    Entertainment|         0.89|           7,533.68|               140.89|                       2.90|               1.00|
|           Sports|         0.95|          14,026.93|               128.45|                       2.55|               0.95|
|     Food & Drink|         1.55|          13,938.62|               320.60|                       2.41|               1.00|
|    Pho

In [99]:
from pyspark.sql.functions import format_number, col

# Let's start off with this. Just grouping by prime_genre and presenting the mean.
group_user_rating_df = df.select('price','rating_count_tot','rating_count_ver','user_rating','user_rating_ver','ver','cont_rating','vpp_lic','prime_genre').groupBy('prime_genre').mean()
group_user_rating_df.show()

# Now that we've calculated the mean, the values for blue-collar and technician are extremely long. 
# We can use format_number to reduce the total amount of decimals. 
# The number two represents the amount of decimals we want to be displayed.
group_user_rating_df = group_user_rating_df.select('user_rating',
                                   format_number('avg(price)',2),
                                   format_number('avg(rating_count_tot)',2),
                                   format_number('avg(rating_count_ver)',2),
                                   format_number('avg(user_rating)',2),
                                   format_number('avg(user_rating_ver)',2),
                                   format_number('avg(vpp_lic)',2))

group_user_rating_df.show()

+-----------+------------------+---------------------+---------------------+----------------+--------------------+------------------+
|user_rating|        avg(price)|avg(rating_count_tot)|avg(rating_count_ver)|avg(user_rating)|avg(user_rating_ver)|      avg(vpp_lic)|
+-----------+------------------+---------------------+---------------------+----------------+--------------------+------------------+
|        0.0|1.0801291711517784|                  0.0|                  0.0|             0.0|                 0.0|0.9795479009687836|
|        3.5| 1.757435897435901|   15503.266381766382|   138.28774928774928|             3.5|   3.158119658119658|0.9928774928774928|
|        4.5| 1.802110401802427|   22029.743522343222|    803.0623357116035|             4.5|   4.199023657529103|0.9981224183251971|
|        2.5|1.1032142857142866|    5419.933673469388|    50.53061224489796|             2.5|  2.2933673469387754|0.9846938775510204|
|        1.0|1.9265909090909084|   29.568181818181817|   17.20